In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import taxcalc as tc
from taxbrain import (
    TaxBrain, differences_plot, distribution_plot, volcano_plot
)
from bokeh.io import output_notebook, show

In [2]:
output_notebook()
pd.options.display.float_format = '{:,.3f}'.format

Loading BokehJS ...

In [12]:
# Set paths to work across Mac/Windows/Linux platforms
# Update cur_path to the path of the repsitory on your local machine
cur_path = "/Users/richardevans/Docs/Economics/OSE/Federal/CharitableContrib"
data_dir = os.path.join(cur_path, 'data')
images_dir = os.path.join(cur_path, 'images')
# Directories to save data
json_dir = os.path.join(cur_path, "json")
tmd_dir = (
    "/Users/richardevans/Docs/Economics/OSE/microsim/" +
    "tax-microdata-benchmarking/tmd/storage/output"
)

## 1. Run Tax-Calculator analysis for first analysis (Mar. 2025)

### 1.1. Setup baseline calculator

In [13]:
# Set up the baseline calculator
base_pol = tc.policy.Policy()
base_pol_dict = base_pol.read_json_reform(os.path.join(json_dir, "ext.json"))
base_pol.implement_reform(base_pol_dict)
gfactors=tc.growfactors.GrowFactors(
    growfactors_filename=os.path.join(tmd_dir, "tmd_growfactors_jason.csv")
)
# rec = tc.records.Records.cps_constructor()
rec = tc.records.Records(
    data=os.path.join(tmd_dir, "tmd_jason.csv.gz"),
    start_year=tc.records.Records.TMDCSV_YEAR,
    gfactors=gfactors,
    weights=os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
    adjust_ratios=None,
)
tc_base = tc.calculator.Calculator(policy=base_pol, records=rec)
print("taxcalc TMD start year is:", tc.records.Records.TMDCSV_YEAR)


taxcalc TMD start year is: 2021


### 1.2. Setup two reforms

#### 1.2.1. Setup reform for permanent $500/$1000 extra deduction (Table 2)

In [14]:
# Create reform 1 calculator
ref1_pol = tc.policy.Policy()
ref1_pol.implement_reform(base_pol_dict)
ref1_pol_dict = base_pol.read_json_reform(
    os.path.join(json_dir, "chddct.json")
)
ref1_pol.implement_reform(ref1_pol_dict)
tc_ref1 = tc.calculator.Calculator(policy=ref1_pol, records=rec)

In [15]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
start_year = 2025
end_year = 2034

rev_base = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
rev_ref1 = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_base.advance_to_year(t)
    tc_base.calc_all()
    rev_base['IIT'][t] = tc_base.weighted_total('iitax')
    rev_base['Payroll'][t] = tc_base.weighted_total('payrolltax')
    rev_base['Sum'][t] = rev_base['IIT'][t] + rev_base['Payroll'][t]
    tc_ref1.advance_to_year(t)
    tc_ref1.calc_all()
    rev_ref1['IIT'][t] = tc_ref1.weighted_total('iitax')
    rev_ref1['Payroll'][t] = tc_ref1.weighted_total('payrolltax')
    rev_ref1['Sum'][t] = rev_ref1['IIT'][t] + rev_ref1['Payroll'][t]

In [9]:
# make table of revenue estimates
rev_base_df = pd.DataFrame.from_dict(rev_base).T
rev_base_df['2025-2034'] = rev_base_df.sum(axis=1)
rev_ref1_df = pd.DataFrame.from_dict(rev_ref1).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff1_df = (rev_ref1_df - rev_base_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff1_df['2025-2034'] = diff1_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_df * 1e-9)
print(rev_ref1_df * 1e-9)
print(diff1_df.to_markdown())

Charitable Act revenue estimates
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2696.659 $2810.901 $2931.118 $3062.111 $3199.674 $3343.232   
Payroll $1706.931 $1786.081 $1860.098 $1931.440 $2005.894 $2082.489 $2160.684   
Sum     $4287.415 $4482.740 $4671.000 $4862.558 $5068.006 $5282.163 $5503.916   

             2032      2033      2034  2025-2034  
IIT     $3492.739 $3645.032 $3801.232 $31563.182  
Payroll $2240.097 $2320.756 $2402.642 $20497.113  
Sum     $5732.836 $5965.789 $6203.873 $52060.295  
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2577.064 $2693.181 $2807.377 $2927.557 $3058.511 $3196.035 $3339.554   
Payroll $1706.931 $1786.081 $1860.098 $1931.440 $2005.894 $2082.489 $2160.684   
Sum     $4283.995 $4479.262 $4667.476 $4858.997 $5064.405 $5278.524 $5500.238   

             2032      2033      2034  
IIT     $3489.026 $3641.283 $3797.449  
Payroll $2240.097 $2320.756 $2402

In [22]:
rev_ref1_df.sum(axis=1) * 1e-9

IIT       $31527.036
Payroll   $20497.113
Sum       $52024.149
dtype: float64

#### 1.2.2. Setup reform for Charitable Act $10,000/$5,000 for 2026 and 2027 (Table 1)

In [18]:
# Create reform 2 calculator
ref2_pol = tc.policy.Policy()
ref2_pol.implement_reform(base_pol_dict)
ref2_pol_dict = base_pol.read_json_reform(
    os.path.join(json_dir, "chddct2.json")
)
ref2_pol.implement_reform(ref2_pol_dict)
tc_ref2 = tc.calculator.Calculator(policy=ref2_pol, records=rec)

In [19]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
rev_ref2 = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_ref2.advance_to_year(t)
    tc_ref2.calc_all()
    rev_ref2['IIT'][t] = tc_ref2.weighted_total('iitax')
    rev_ref2['Payroll'][t] = tc_ref2.weighted_total('payrolltax')
    rev_ref2['Sum'][t] = rev_ref2['IIT'][t] + rev_ref2['Payroll'][t]

In [20]:
# make table of revenue estimates
rev_ref2_df = pd.DataFrame.from_dict(rev_ref2).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff2_df = (rev_ref2_df - rev_base_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff2_df['2025-2034'] = diff2_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_df * 1e-9)
print(rev_ref2_df * 1e-9)
print(diff2_df.to_markdown())

Charitable Act revenue estimates
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2696.659 $2810.901 $2931.118 $3062.111 $3199.674 $3343.232   
Payroll $1706.931 $1786.081 $1860.098 $1931.440 $2005.894 $2082.489 $2160.684   
Sum     $4287.415 $4482.740 $4671.000 $4862.558 $5068.006 $5282.163 $5503.916   

             2032      2033      2034  2025-2034  
IIT     $3492.739 $3645.032 $3801.232 $31563.182  
Payroll $2240.097 $2320.756 $2402.642 $20497.113  
Sum     $5732.836 $5965.789 $6203.873 $52060.295  
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2680.868 $2794.657 $2931.118 $3062.111 $3199.674 $3343.232   
Payroll $1706.931 $1786.081 $1860.098 $1931.440 $2005.894 $2082.489 $2160.684   
Sum     $4287.415 $4466.950 $4654.755 $4862.558 $5068.006 $5282.163 $5503.916   

             2032      2033      2034  
IIT     $3492.739 $3645.032 $3801.232  
Payroll $2240.097 $2320.756 $2402

In [21]:
rev_ref2_df.sum(axis=1) * 1e-9

IIT       $31531.147
Payroll   $20497.113
Sum       $52028.260
dtype: float64

## 2. Run Tax-Calculator analysis for second analysis (June 2025)

### 2.1. Run reform of $150/$300 extra deduction with a 0.5% AGI floor for itemized charitable deduction

### 2.2. Run reform of $150/$300 extra deduction with a 1.0% AGI floor for itemized charitable deduction

### 2.3. Run reform of $500/$1,000 extra deduction with a 0.5% AGI floor for itemized charitable deduction

### 2.4. Run reform of $500/$1,000 extra deduction with a 1.0% AGI floor for itemized charitable deduction

### 2.5. Run reform of $5,000/$10,000 extra deduction with a 0.5% AGI floor for itemized charitable deduction

### 2.6. Run reform of $5,000/$10,000 extra deduction with a 1.0% AGI floor for itemized charitable deduction